In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import tensorflow as tf
import numpy as np

In [3]:
model_path = '../gdrive/MyDrive/Colab_modules/my_model_keras'
model = tf.saved_model.load(model_path)

In [12]:
test_path = "../gdrive/MyDrive/Colab_data/dog-cat-full-dataset-master/data/test"
IMG_SIZE = 160

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_path, image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True, seed=42, batch_size=32
)

test_dataset_no_batch = tf.keras.utils.image_dataset_from_directory(
    test_path, image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True, seed=1
)

Found 5000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [13]:
def format_sample(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    return image, label

test_dataset = test_dataset.map(format_sample)
test_dataset_no_batch = test_dataset.map(format_sample)

In [17]:
class_names = ['cats', 'dogs']

idx = 0
result = []
for images, datasset_labels in test_dataset_no_batch:
  print("idx:", idx)
  predictions = model(images, training=False)
  probs = tf.nn.softmax(predictions)
  print("probs:", probs)
  class_indexes = tf.argmax(probs, axis = 1 ).numpy()
  print("class_indexes:", class_indexes)
  print("datasset_labels:", datasset_labels.numpy())
  idx += 1
  if idx > 1:
    break

idx: 0
probs: tf.Tensor(
[[0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]], shape=(32, 1), dtype=float32)
class_indexes: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
datasset_labels: [1 1 0 1 0 0 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 0 0 0 1 0 0 1 0 0 1 0]
idx: 1
probs: tf.Tensor(
[[0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994]
 [0.99999994